# Diabetes Prediction with Deep Learning

### Machine Learning step by step:
   1. Business & Data Understanding
   2. Data Cleaning & Analysis
   3. Data Modeling
   4. Model Evaluation
   5. Model Deployment & Maintenance

## Import necessary libraries

In [ ]:
!pip install bayesian-optimization

#### Base Libraries

In [13]:
import pandas as pd
import numpy as np 
import os
from scipy import stats
from scipy.stats import skew

### Plot Style using Seaborn

In [ ]:
plt.style.use('seaborn')

## 1. Business & Data Understanding

### This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. 
The objective of this notebook is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. 
Several constraints were placed on the selection of these instances from a larger database. 
In particular, all patients here are females at least 21 years old of Pima Indian heritage.

### Load dataset

In [ ]:
column_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness','Insulin','BMI','PedigreeFunction','Age','y']
df_diabetes = pd.read_csv("C:/Users/ejhas/OneDrive/Documents/miracles/JOBS/AI Bootcamp/WEEK6/day1/Dataset/pima-indians-diabetes.csv", header=None, names=column_names)

In [ ]:
df_diabetes.info()

In [ ]:
print('Total Rows : ', df_diabetes.shape[0])
print('Total Columns/Features : ', df_diabetes.shape[1])

In [ ]:
df_diabetes

In [ ]:
df_diabetes.hist(figsize=(9, 9))

In [ ]:
for x in df_diabetes.columns:
    sns.displot(df_diabetes[x],kde=True)
plt.show()

## 2. Data Cleaning & Analysis

#### Missing or Null Data points

In [ ]:
# Missing value percentage checking
missing_percentage = (df_diabetes.isnull().sum() / len(df_diabetes)) * 100

# Show the missing percentage
print(missing_percentage)

#### Unexpected Outliers Identified from Histogram
There are three features that could probably have outliers, such as Glucose, Blood Pressure, BMI, Insulin, Skin Thickness, Pedigree Function

#### Identify the columns with the value "0" by creating a boolean mask

In [ ]:
# Exclude the "y" column
columns_to_check = df_diabetes.columns.difference(["y"])

# Create a boolean mask for rows where the value is "0"
mask = (df_diabetes[columns_to_check] == 0)

#### Summarize the results to see which columns have the value "0" in them

In [ ]:
# Summarize the boolean mask to check which columns have at least one "0" value
columns_with_zeros = mask.any()

# List the column names with "0" values
columns_with_zeros = columns_with_zeros[columns_with_zeros].index

print("Columns with '0' values:")
print(columns_with_zeros)

#### Counts the number of zero values

In [ ]:
# Exclude the 'y' column from the selection
numeric_columns = df_diabetes.select_dtypes(include=['number']).drop(columns=['y'])

# Count the number of zeros in each selected column
zero_counts = numeric_columns.eq(0).sum()

# Print the columns with zero counts
print("Columns with zero values and their counts (excluding 'y'):")
print(zero_counts[zero_counts > 0])

#### Target Value Counts

In [ ]:
df_diabetes['y'].value_counts()

In [ ]:
sns.countplot(x='y', data=df_diabetes)
plt.show()

In [ ]:
#df_diabetes['y'].value_counts().plot(kind='pie')
plt.pie(df_diabetes['y'].value_counts().tolist(), labels=['0', '1'], autopct='%1.1f%%')

plt.title("Outcomes")
plt.tight_layout()

plt.show()

#### Correlation

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
corr = df_diabetes.corr()
sns.heatmap(corr[(corr >= 0.5) | (corr <= -0.4)], 
            ax=ax,
            mask=np.triu(corr, k=0),
            cmap='viridis',
            vmax=1.0, 
            vmin=-1.0, 
            linewidths=1.0,
            annot=True,
            annot_kws={'fontsize': 20},
            square=True,
            fmt='.3f')

plt.show()

In [ ]:
df_diabetes.corr()['y']

##### Diabetes Percentage by Age Range

In [ ]:
# Define Age ranges
age_bins = [20, 30, 40, 50, 60, 70, 80]  # Adjust the age ranges as needed
age_labels = ['20-29', '30-39', '40-49', '50-59', '60-69', '70+']

#Use the pd.cut function to categorize ages into age ranges:
df_diabetes['AgeRange'] = pd.cut(df_diabetes['Age'], bins=age_bins, labels=age_labels, include_lowest=True)

#Group the data by age range and calculate the percentage of diabetes cases (Diabetes=1):
age_diabetes = df_diabetes.groupby('AgeRange')['y'].mean() * 100

#Plot the data:
plt.bar(age_diabetes.index, age_diabetes)
plt.xlabel("Age Range")
plt.ylabel("Percentage with Diabetes")
plt.title("Diabetes Percentage by Age Range")
plt.show()

#### Check BMI distribution

In [ ]:
lower_limit = 21
upper_limit = 80
#Filter the DataFrame to select only the rows within your specified range:

filtered_df = df_diabetes[(df_diabetes['BMI'] >= lower_limit) & (df_diabetes['BMI'] <= upper_limit)]

plt.hist(filtered_df['BMI'], bins=50, edgecolor='k')  # You can adjust the number of bins.
plt.xlabel('BMI')
plt.ylabel('Frequency')
plt.title(f'Histogram of BMI Values in Range {lower_limit}-{upper_limit}')
plt.grid(True)
plt.show()

##### BMI vs. Diabetes

In [ ]:
# Define BMI ranges
bmi_ranges = [0, 18.5, 24.9, 29.9, 34.9, 100]
bmi_labels = ['Underweight', 'Normal', 'Overweight', 'Obese Class I', 'Obese Class II+']

#Use the pd.cut function to categorize ages into age ranges:
df_diabetes['BMIRange'] = pd.cut(df_diabetes['BMI'], bins=bmi_ranges, labels=bmi_labels, include_lowest=True)

#Group the data by age range and calculate the percentage of diabetes cases (Diabetes=1):
bmi_diabetes = df_diabetes.groupby('BMIRange')['y'].mean() * 100

#Plot the data:
plt.bar(bmi_diabetes.index, bmi_diabetes)
plt.xlabel("BMI Range")
plt.ylabel("Percentage with Diabetes")
plt.title("Diabetes Percentage by BMI Range")
plt.show()

##### Glucose vs Diabetes

In [ ]:
#Create two separate data series for diabetic and non-diabetic patients:
glucose_diabetic = df_diabetes[df_diabetes['y'] == 1]['Glucose']

glucose_non_diabetic = df_diabetes[df_diabetes['y'] == 0]['Glucose']
# Define the bins (ranges) for the histogram
bins = [50, 70, 100, 125, 150, 200, 250]

# Plot histogram for diabetic patients
plt.hist(glucose_diabetic, bins=bins, alpha=0.5, label='Diabetic', color='red')

# Plot histogram for non-diabetic patients
plt.hist(glucose_non_diabetic, bins=bins, alpha=0.5, label='Non-Diabetic', color='blue')

# Labeling and legend
plt.xlabel('Glucose Level')
plt.ylabel('Frequency')
plt.title('Glucose Distribution by Diabetes Status')
plt.legend()

# Show the plot
plt.show()

#### Drop zero values
Since we have 2 columns with too much zero values, we then need to drop:
1. SkinThickness    227
2. Insulin          374

In [ ]:
columns_to_drop = ['SkinThickness', 'Insulin', 'AgeRange', 'BMIRange']
df_new = df_diabetes.drop(columns_to_drop, axis=1)

In [ ]:
df_new.head()

#### Treating the rest of columns with zero values
Glucose has normal distributions, we replace 0 values in those columns by mean value. BMI and Blood Pressure have negative skewed distributions, median will be used to replace them.
1. Pregnancies      111
2. Glucose            5
3. BloodPressure     35
4. BMI               11

In [ ]:
# Calculate the skewness for all numerical columns
skewness = df_new.skew()

# Print the skewness values
print(skewness)

In [ ]:
# Calculate mean and median (excluding zeros)
mean = df_new[df_new['Glucose'] != 0]['Glucose'].mean()
median_bmi = df_new[df_new['BMI'] != 0]['BMI'].median()
median_bp = df_new[df_new['BloodPressure'] != 0]['BloodPressure'].median()

In [ ]:
# Replace zeros on Glucose with mean
df_new['Glucose'].replace(0, mean, inplace=True)

# Replace zeros on BMI with median
df_new['BMI'].replace(0, median_bmi, inplace=True)

# Replace zeros on BloodPressure with median
df_new['BloodPressure'].replace(0, median_bp, inplace=True)

In [ ]:
df_new.eq(0).sum()

## 3. Data Modelling


### Split Datasets

In [ ]:
df_new.head()

In [ ]:
features = df_new.values[:,:6]
print(features.shape)

In [ ]:
labels = df_new.values[:,6:]
print(labels.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    labels, 
                                                    test_size=0.2, 
                                                    random_state=77)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

### Define Model

In [ ]:
# Define a function to create Keras model
def create_model(optimizer='adam'):
    model = Sequential()
    model.add(Dense(8, input_dim=6, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
# Create a KerasClassifier based on  model function
model = KerasClassifier(build_fn=create_model, epochs=140, batch_size=7, verbose=0)

In [ ]:
# Define hyperparameters for tuning
param_grid = {
    'optimizer': ['adam', 'sgd'],
    # Add more hyperparameters as needed
}

In [ ]:
# Define the number of cross-validation folds (e.g., 5)
n_splits = 5
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [ ]:
# Create a GridSearchCV instance
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=cv, n_jobs=1)

In [ ]:
# Fit the GridSearchCV on your training data
grid_result = grid.fit(X_train, y_train)

In [ ]:
# Print the best parameters and results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))